# Proposed code to calculate the gear position relative to the robot camera given processed data from the camera and Raspberry Pi

In [1]:
import math

## Some constants

In [3]:
# reflective tape dimensions - all in inches
tape_height_inches = 5.0
tape_width_inches = 2.
tape_centers_delta_inches = 8.25 # distance between centers of the tape strips measured horizontally

# camera constants - for now we are using the camera field of view as is. No calibration yet
# From the specs for the Microsoft LifeCam HD 3000
#    https://www.microsoft.com/accessories/en-us/products/webcams/lifecam-hd-3000/t3h-00011#techspecs-connect
camera_field_of_view = 70.0 # degrees - sort of the angle subtended
camera_width_pixels = 1280.
camera_height_pixels = 800.

## Output from the vision processing pipeline on the Raspberry Pi

These are in the pixel coordinate system of the camera image with (0,0) in the center of the image

    - x_center_left_tape_pixels
    - height_left_tape_pixels

    - x_center_right_tape_pixels
    - height_right_tape_pixels

## Function to calculate the position of the gear peg target given information about the location and sizes of the reflective tape in the image.

Output of the function is in the coordinate system of the camera itself where the y axis points out along the camera lens axis and the x axis points to the right

In [14]:
def calc_gear_peg_location(x_center_left_tape_pixels, height_left_tape_pixels, 
                           x_center_right_tape_pixels, height_right_tape_pixels):
    '''
    Given:
        - location of left and right tape centers in camera pixel coordinates
        - height of left and right tapes in camera pixel coordinates
    Calculate:
        - the location of the gear peg in the (x,y) coordinates of the camera, in inches    
    '''
    x_center_left_tape, y_center_left_tape, x_center_right_tape, y_center_right_tape = \
        calc_tape_locations(x_center_left_tape_pixels, height_left_tape_pixels, 
                        x_center_right_tape_pixels, height_right_tape_pixels)
        
    # The gear peg is in the middle between the two reflective tapes
    x_target_center = (x_center_right_tape + x_center_left_tape) * 0.5
    y_target_center = (y_center_right_tape + y_center_left_tape) * 0.5
    
    return x_target_center, y_target_center

## Need some supporting functions

In [15]:
def calc_tape_locations(x_center_left_tape_pixels, height_left_tape_pixels, 
                        x_center_right_tape_pixels, height_right_tape_pixels):
    '''
    Given:
        - location of left and right tape centers in camera pixel coordinates
        - height of left and right tapes in camera pixel coordinates
    Calculate:
        - the locations of the centers of the tape in the (x,y) coordinates of the camera, in inches
    '''
    # calculate the distance from the two pieces of tape to the robot camera
    tape_left_distance = calc_distance_from_camera_from_heights( height_left_tape_pixels, tape_height_inches)
    tape_right_distance = calc_distance_from_camera_from_heights( height_right_tape_pixels, tape_height_inches)
    
    x_center_left_tape, y_center_left_tape = calc_xy_tape_center(tape_left_distance, x_center_left_tape_pixels)
    x_center_right_tape, y_center_right_tape = calc_xy_tape_center(tape_right_distance, x_center_right_tape_pixels)
    
    return x_center_left_tape, y_center_left_tape, x_center_right_tape, y_center_right_tape

In [16]:
def calc_distance_from_camera_from_heights( height_pixels, height_inches):
    '''
    Given:
        - the height of one of the pieces of tape in pixels in the image
        - the actual height of the piece of tape in inches
    Calculate:
        - the distance from the center of the camera coordinate system to the tape
    '''
    height_angle_subtended = camera_field_of_view * height_pixels / camera_height_pixels
    distance_from_camera = height_inches / math.tan(math.radians(height_angle_subtended))
    return distance_from_camera

In [17]:
def calc_xy_tape_center(tape_distance_from_camera, x_center_tape_pixels):
    '''
    Given:
        - distance in inches from the center of the camera coordinate system to the tape
        - the location of the center of the tape in the x coordinate of the camera image, in pixels
    Calculate:
        - the location of the tape in the (x,y) coordinate system of the camera, in inches
    '''
    x_center_tape = tape_distance_from_camera * math.sin(math.radians(x_center_tape_pixels * camera_field_of_view / camera_width_pixels))
    y_center_tape = tape_distance_from_camera * math.cos(math.radians(x_center_tape_pixels * camera_field_of_view / camera_width_pixels))
    return x_center_tape, y_center_tape

## Example of using calc_tape_locations function

In [18]:
x_center_left_tape_pixels = 0.0
height_left_tape_pixels = 134.49473065166453

x_center_right_tape_pixels = 202.3121080248823
height_right_tape_pixels = 106.74787765397747

In [20]:
x_target_center, y_target_center = calc_gear_peg_location(x_center_left_tape_pixels, height_left_tape_pixels, 
                           x_center_right_tape_pixels, height_right_tape_pixels)
x_target_center, y_target_center

(2.9168154723945077, 26.916815472394507)

#### quick little test of sensitivity of calculation if the measured values from the image processing are all off by one pixel

In [21]:
x_target_center, y_target_center = calc_gear_peg_location(x_center_left_tape_pixels + 1.0, height_left_tape_pixels + 1.0, 
                           x_center_right_tape_pixels + 1.0, height_right_tape_pixels + 1.0)
x_target_center, y_target_center

(2.9147254695444267, 26.681993771273106)

#### Looks like not much, which is good